## Preparing the environment

In [1]:
#Importing libraries
import json
import requests
from math import radians, sin, cos, sqrt, atan2
import pandas as pd 
import folium as f
import numpy as np
from shapely.geometry import Point, Polygon
from folium.plugins import BeautifyIcon
import openrouteservice as ors
from openrouteservice import convert
from datetime import datetime,timedelta
import random
from faker import Faker #faker a library for fake data generation
from faker.providers import BaseProvider
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Building the experience with OOP

class CC:
    def __init__(self,location):
        self.location = location
        
class TA: #Transporitng Agent
    def __init__(self,capacity,id_): #capacity in .kg
        self.capacity = capacity
        self.id=id_
    def get_location():
        pass
    def curr_capacity():
        pass
    

class GP: #garbage provider
    def __init__(self,id_,point,name,fname,phone,email,password):
        self.name=name
        self.fname=fname
        self.id = id_
        self.point = point
        self.phone = phone
        self.email = email
        self.password=password
        self.created_at= datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
    def client_location(self,id_,adresse,latitude,longitude,ex):
        new = pd.DataFrame({'id' : [id_],
                              'id_client' : [self.id],
                  'latitude': [latitude],
                'longitude': [longitude],
               'adresse': [adresse],
                'created_at': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')] })
        ex.dl = pd.concat([ex.dl , new],ignore_index = True)
        
    def request(self,id_,id_client,start,finish,location,ex): #quantity in .kg
        new = pd.DataFrame({'id':[id_],
                              'id_client':[id_client],
                              'id_livreur':None,
                              'date': [start],
                              'point': [self.point],
                              'status':0,
                  'Open_From' : [start],
                  'Open_To': [finish],
               'location': [location],
                'created_at': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')]})        
        ex.db = pd.concat([ex.db,new],ignore_index = True)
        
        

class experience:
    def __init__(self):
        d = {'id':[],'id_client':[],'id_livreur':[],'date':[],'point':[],'status':[],'Open_From':[],'Open_To':[],'location':[],'created_at':[]}
        d_location = {'id':[],'id_client':[] ,'adresse':[],'latitude':[],'longitude':[],'created_at':[]}
        d_client = {'id':[],'name':[],'fname':[],'phone':[],'email':[],'password':[],'point':[],'created_at':[]} 
        d_colis = {'id':[],'id_dechet':[],'id_request':[],'poids_estimee':[],'poids_reel':[] }
        
        self.db=pd.DataFrame.from_dict(d)
        self.dl=pd.DataFrame.from_dict(d_location)
        self.dc=pd.DataFrame.from_dict(d_client)
        self.dcolis = pd.DataFrame.from_dict(d_colis)
        
        self.gps = []
        self.tas = []
        self.cc = []
        
    def add_gp(self,id_,point,name,fname,phone,email,password):
        new_gp = GP(id_,point,name,fname,phone,email,password)
        new = pd.DataFrame({'id':[new_gp.id],'name':[new_gp.name],'fname':[new_gp.fname],'phone':[new_gp.phone],'email':[new_gp.email],'password':[new_gp.password],'point':[new_gp.point],'created_at':[new_gp.created_at]})
        self.dc = pd.concat([self.dc,new],ignore_index = True)
        self.gps.append(new_gp)

    def add_tas(self,quantity,id_):
        new_ta = TA(10,id_)
        self.tas.append(new_ta)

In [3]:
#generating random dates for the dataset.
def time_generator():
    start=random.randint(6,19)
    end=random.randint(start+1,20)
    start_date=datetime(2023,8,1,start)
    finish_date=datetime(2023,8,1,end)
    start_date=start_date.strftime("%Y-%m-%d %H:%M:%S")
    finish_date=finish_date.strftime("%Y-%m-%d %H:%M:%S")
    start_date=datetime.strptime(start_date,"%Y-%m-%d %H:%M:%S")
    finish_date=d=datetime.strptime(finish_date,"%Y-%m-%d %H:%M:%S")
    return start_date,finish_date


In [4]:
#calculating distance between two points
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    earth_radius = 6371000  
    distance = earth_radius * c
    return distance

In [5]:
# Create a Folium map
m = f.Map(location=list(reversed([10.76049638984947,34.73963348132654])), tiles="cartodbpositron", zoom_start=10)
def generate_locations(number_points):
    requests=[]
    for i in range(6):
        requests.append(number_points*sfax_data["Portions"][i])

    polygon = sfax_data["geometry"][0]
    random_points = []
    all_points=[]

    for i,j in enumerate(requests):
        polygon = sfax_data["geometry"][i]
            # Repeat the generation process until you have the desired number of points
        while len(random_points) < j:
                # Generate a random point within the bounding box of the polygon
            point = Point(np.random.uniform(polygon.bounds[0], polygon.bounds[2]),
                    np.random.uniform(polygon.bounds[1], polygon.bounds[3])
                )

                # Check if the point is within the polygon
            if point.within(polygon):
                random_points.append(point)
                all_points.append(point)
            # Add the generated points to the list
        random_points.append(point)
        random_points=[]
        
    return all_points

In [6]:
# Create a Faker instance
fake = Faker('')

# Generate a fake email address
fake_email = fake.email(domain='gmail.com')
fake_name=fake.first_name()
fake_lastname=fake.last_name()
class TunisianProvider(BaseProvider):
    def phone_number_tn(self):
        return "+216 " + self.random_element(["2", "5", "7"]) + self.numerify('#######')

# Create a Faker instance with the custom Tunisian provider
fake_tn = Faker()
fake_tn.add_provider(TunisianProvider)

# Generate fake phone number for Tunisia
fake_phone_tn = fake_tn.phone_number_tn()

In [9]:
# Loading sfax Delegations

sfax_delegations=gpd.read_file('delegations-full.shx')
sfax=[2,56,83,84,251,253]
sfax_data=sfax_delegations.iloc[sfax]
sfax_data.index=[0,1,2,3,4,5]
delegations=["Thyna","Sfax_Ouest","Sakiet_Dayer","Sakiet_Ziet","Sfax_Medina","Sfax_Sud"]
portions=[0.12,0.176,0.194,0.154,0.152,0.2]
sfax_data.insert(0, "Delegation", delegations, True)
sfax_data.insert(2,"Portions",portions,True)

In [10]:
display(sfax_data)

,Delegation,geometry,Portions
0,Thyna,"POLYGON ((10.62949 34.75983, 10.71175 34.77413...",0.120
1,Sfax_Ouest,"POLYGON ((10.74264 34.71875, 10.74264 34.71863...",0.176
2,Sakiet_Dayer,"POLYGON ((10.81274 34.89964, 10.82456 34.89283...",0.194
3,Sakiet_Ziet,"POLYGON ((10.74854 34.82049, 10.73527 34.78593...",0.154
4,Sfax_Medina,"MULTIPOLYGON (((10.78125 34.72986, 10.78125 34...",0.152
5,Sfax_Sud,"POLYGON ((10.71175 34.77413, 10.62949 34.75983...",0.200


## Running the environment

In [11]:
ideal_experience=experience()

In [13]:
number_points=int(input("Specify Test set of numbers: "))

Specify Test set of numbers:  30


### Creating Databases

In [14]:
#generating clients

for i in range(1000):
    first_name = fake.first_name()
    last_name = fake.last_name()
    mail = first_name+last_name+str(random.randint(0,12)) + '@gmail.com'
    ideal_experience.add_gp(int(i),random.randint(0,100),first_name,last_name,fake_tn.phone_number_tn(),mail,random.randint(100000,2000000))
ideal_experience.dc.id=pd.to_numeric(ideal_experience.dc.id,downcast='integer')
ideal_experience.dc.point=pd.to_numeric(ideal_experience.dc.point,downcast='integer')
ideal_experience.dc.password=pd.to_numeric(ideal_experience.dc.password,downcast='integer')

In [15]:
ideal_experience.dc

,id,name,fname,phone,email,password,point,created_at
0,0,Laura,Barr,+216 75174406,LauraBarr2@gmail.com,404503,14,2025-03-19 08:32:01
1,1,Mary,Morris,+216 28666598,MaryMorris4@gmail.com,674073,52,2025-03-19 08:32:01
2,2,Isaac,Williams,+216 51991437,IsaacWilliams3@gmail.com,1630676,12,2025-03-19 08:32:01
3,3,Erica,Bennett,+216 56367569,EricaBennett8@gmail.com,1028348,3,2025-03-19 08:32:01
4,4,Mallory,Gallegos,+216 79907787,MalloryGallegos12@gmail.com,214595,60,2025-03-19 08:32:01
...,...,...,...,...,...,...,...,...
995,995,Joe,Flores,+216 77319790,JoeFlores11@gmail.com,1380887,98,2025-03-19 08:32:02
996,996,Alexander,Hester,+216 77618097,AlexanderHester9@gmail.com,845428,3,2025-03-19 08:32:02
997,997,Vicki,Young,+216 57795624,VickiYoung6@gmail.com,319649,5,2025-03-19 08:32:02
998,998,Rachel,Jones,+216 53406338,RachelJones5@gmail.com,1448281,37,2025-03-19 08:32:02


In [16]:
#Generating Client Locations

sfax=['merkez chihya','Merkez Bouacida','Cité El Habib','Sidi Abbes','Sfax Jadida','merkez el alia','Sfax 15 Novembre','Cité Khayri','Cité Bahri','Esskhira','Sfax','Sfax Hached','Sakiet Eddaier','tyna','Sakiet Ezzit','Sfax Maghreb Arabe']
locations = generate_locations(number_points) 
for i in range(number_points):
    random_id=random.randint(0,999)
    random_adress=random.randint(0,15)
    ideal_experience.gps[random_id].client_location(i,sfax[random_adress],locations[i].y,locations[i].x,ideal_experience)
ideal_experience.dl.id=pd.to_numeric(ideal_experience.dl.id,downcast='integer')
ideal_experience.dl.id_client=pd.to_numeric(ideal_experience.dl.id_client,downcast='integer')

In [17]:
display(ideal_experience.dl.head())

,id,id_client,adresse,latitude,longitude,created_at
0,0,186,Sfax,34.725608,10.690784,2025-03-19 08:32:07
1,1,631,Sfax Maghreb Arabe,34.666823,10.651204,2025-03-19 08:32:07
2,2,131,Sfax 15 Novembre,34.688431,10.727914,2025-03-19 08:32:07
3,3,899,tyna,34.707125,10.688564,2025-03-19 08:32:07
4,4,138,merkez el alia,34.741278,10.745410,2025-03-19 08:32:07


In [18]:
#Generating Requests

for i in range(number_points):
    id_client = ideal_experience.dl[i:i+1].id_client
    location = str(float(ideal_experience.dl[i:i+1].latitude)) + '  ' + str(float(ideal_experience.dl[i:i+1].longitude))
    start,finish=time_generator()
    ideal_experience.gps[int(id_client)].request(i,int(id_client),start,finish,location,ideal_experience)
    
ideal_experience.db.id=pd.to_numeric(ideal_experience.db.id,downcast='integer')
ideal_experience.db.id_client=pd.to_numeric(ideal_experience.db.id_client,downcast='integer')
ideal_experience.db.point=pd.to_numeric(ideal_experience.db.point,downcast='integer')
ideal_experience.db.status=pd.to_numeric(ideal_experience.db.status,downcast='integer')

In [19]:
display(ideal_experience.db.head())

,id,id_client,id_livreur,date,point,status,Open_From,Open_To,location,created_at
0,0,186,None,2023-08-01 16:00:00,28,0,2023-08-01 16:00:00,2023-08-01 20:00:00,34.72560838249331 10.690784313871676,2025-03-19 08:32:08
1,1,631,None,2023-08-01 13:00:00,8,0,2023-08-01 13:00:00,2023-08-01 19:00:00,34.66682292137911 10.651203642067644,2025-03-19 08:32:08
2,2,131,None,2023-08-01 14:00:00,16,0,2023-08-01 14:00:00,2023-08-01 15:00:00,34.68843091112631 10.727914400426934,2025-03-19 08:32:08
3,3,899,None,2023-08-01 12:00:00,32,0,2023-08-01 12:00:00,2023-08-01 18:00:00,34.707124698904195 10.68856401096503,2025-03-19 08:32:08
4,4,138,None,2023-08-01 06:00:00,88,0,2023-08-01 06:00:00,2023-08-01 16:00:00,34.741278025657735 10.745409566146613,2025-03-19 08:32:08


In [20]:
#Generating des colis
c=0
for i in range(number_points):
    number_colis = random.randint(1,3)
    for j in range(number_colis):
        id_request = int(ideal_experience.db[i:i+1].id)
        new = pd.DataFrame({'id' : [c],'id_dechet' : [random.randint(0,10)],'id_request': [id_request],'poids_estimee':[random.randint(1,5)],'poids_reel':None})
        ideal_experience.dcolis=pd.concat([ideal_experience.dcolis,new],ignore_index=True)
        c+=1
    
ideal_experience.dcolis.id=pd.to_numeric(ideal_experience.dcolis.id,downcast='integer')
ideal_experience.dcolis.id_dechet=pd.to_numeric(ideal_experience.dcolis.id_dechet,downcast='integer')
ideal_experience.dcolis.id_request=pd.to_numeric(ideal_experience.dcolis.id_request,downcast='integer')
ideal_experience.dcolis.poids_estimee=pd.to_numeric(ideal_experience.dcolis.poids_estimee,downcast='integer')

In [21]:
display(ideal_experience.dcolis.head())

,id,id_dechet,id_request,poids_estimee,poids_reel
0,0,1,0,5,None
1,1,3,0,5,None
2,2,8,0,5,None
3,3,2,1,4,None
4,4,0,1,5,None


In [22]:
quantities_per_request=[] #new column will be added to the database of requests that will later be entered into the optimisation process.
for i in range(number_points):
    quantities_per_request.append(sum(ideal_experience.dcolis[ideal_experience.dcolis.id_request==i].poids_estimee))

In [23]:
ideal_experience.db.insert(9,'quantity',quantities_per_request) #adding new column

## Visualizing 

### Visualizing Sfax delegations

In [24]:
# Create a Folium map
m = f.Map(location=list(reversed([ 10.76049638984947,34.73963348132654])), tiles="cartodbpositron", zoom_start=10)

# Define your polygon geometry
for i in range(6):
    polygon_geometry = sfax_data[i:i+1]['geometry']

    # Convert the polygon to a GeoJSON-like dictionary
    polygon_geojson = polygon_geometry.__geo_interface__

    # Create a GeoJson object from the polygon GeoJSON-like dictionary
    geojson_polygon = f.GeoJson(polygon_geojson)
    
    # Add the GeoJson polygon to the map
    geojson_polygon.add_to(m)

In [25]:
display(m)

### Representation for an optimization process.

In [26]:
map_ = f.Map(location=[34.739493943787394, 10.760167212062342], tiles='cartodbpositron', zoom_start=11)
for i in range(6):
    polygon_geometry = sfax_data[i:i+1]['geometry']

    # Convert the polygon to a GeoJSON-like dictionary
    polygon_geojson = polygon_geometry.__geo_interface__

    # Create a GeoJson object from the polygon GeoJSON-like dictionary
    geojson_polygon = f.GeoJson(polygon_geojson)
    
    # Add the GeoJson polygon to the map
    geojson_polygon.add_to(map_)
c = 0
for location in ideal_experience.dl.itertuples():
    quantity=random.randint(1,5)
    tooltip = f.map.Tooltip("<h4><b>ID {}</b></p><p>Quantity : <b>{}</b></p>".format( #the quantity is random for now
        location.Index, quantities_per_request[c]
    ))
    c += 1
    f.Marker(
        location=[location.latitude, location.longitude],
        tooltip=tooltip,
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(location.Index),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(map_)

In [27]:
ideal_experience.db.to_csv('RequestsDatabase.csv')

In [28]:
ideal_experience.dc

,id,name,fname,phone,email,password,point,created_at
0,0,Laura,Barr,+216 75174406,LauraBarr2@gmail.com,404503,14,2025-03-19 08:32:01
1,1,Mary,Morris,+216 28666598,MaryMorris4@gmail.com,674073,52,2025-03-19 08:32:01
2,2,Isaac,Williams,+216 51991437,IsaacWilliams3@gmail.com,1630676,12,2025-03-19 08:32:01
3,3,Erica,Bennett,+216 56367569,EricaBennett8@gmail.com,1028348,3,2025-03-19 08:32:01
4,4,Mallory,Gallegos,+216 79907787,MalloryGallegos12@gmail.com,214595,60,2025-03-19 08:32:01
...,...,...,...,...,...,...,...,...
995,995,Joe,Flores,+216 77319790,JoeFlores11@gmail.com,1380887,98,2025-03-19 08:32:02
996,996,Alexander,Hester,+216 77618097,AlexanderHester9@gmail.com,845428,3,2025-03-19 08:32:02
997,997,Vicki,Young,+216 57795624,VickiYoung6@gmail.com,319649,5,2025-03-19 08:32:02
998,998,Rachel,Jones,+216 53406338,RachelJones5@gmail.com,1448281,37,2025-03-19 08:32:02


In [29]:
depot=[34.75093585415626,10.777167937186457]
f.Marker(
    location=depot,
    icon=f.Icon(color="green", icon="bus", prefix='fa'),
    setZIndexOffset=1000
).add_to(map_)

display(map_)

### Using Openrouteservice API for Optimization

In [30]:
vehicles=list()
for idx in range(3):
    vehicles.append(
        ors.optimization.Vehicle(
            id=idx,
            start=list(reversed(depot)),
            # end=list(reversed(depot)),
            capacity=[300],
            time_window=[1690866000, 1690916400] , # Fri 8-20:00, expressed in POSIX timestamp
        )
    )

In [31]:
#test=pd.read_csv("Request2.csv")
#test.drop(["Unnamed: 0"],axis=1,inplace=True)
#ideal_experience.db=test
#open_from=[datetime.strptime(i, '%Y-%m-%d %H:%M:%S') for i in ideal_experience.db.Open_From]
#open_to=[datetime.strptime(i, '%Y-%m-%d %H:%M:%S') for i in ideal_experience.db.Open_To]
#ideal_experience.db.Open_From=open_from
#ideal_experience.db.Open_To=open_to

In [32]:
deliveries = list()
for delivery in ideal_experience.db.itertuples():
    deliveries.append(
        ors.optimization.Job(
            id=delivery.Index,
            location=[float(delivery.location.split('  ')[1]), float(delivery.location.split('  ')[0])],
            service=1200,  # Assume 20 minutes at each site
            amount=[delivery.quantity],
            time_windows=[[
                int(delivery.Open_From.timestamp()),  # VROOM expects UNIX timestamp
                int(delivery.Open_To.timestamp())
            ]]
        )
    )

In [33]:
ors_client = ors.Client(key='YOUR-KEY')  # Get an API key from https://openrouteservice.org/dev/#/signup
result = ors_client.optimization(
    jobs=deliveries,
    vehicles=vehicles,
    geometry=True
)


In [34]:
for color, route in zip(['green', 'red', 'blue'], result['routes']):
    decoded = ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
    gj = f.GeoJson(
        name='Vehicle {}'.format(route['vehicle']),
        data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                         "geometry": decoded,
                                                         "properties": {"color": color}
                                                         }]},
        style_function=lambda x: {"color": x['properties']['color']}
    )
    gj.add_child(f.Tooltip(
        """<h4>Vehicle {vehicle}</h4>
        <b>Distance</b> {distance} m <br>
        <b>Duration</b> {duration} secs
        """.format(**route)
    ))
    gj.add_to(map_)

f.LayerControl().add_to(map_)
display(map_)

### Display Agents Schedule

In [35]:
extract_fields = ['distance', 'amount', 'duration']
data = [{key: route[key] for key in extract_fields} for route in result['routes']]

vehicles_df = pd.DataFrame(data)
vehicles_df.index.name = 'vehicle'
print("The Recap: ")
print()
display(vehicles_df)

The Recap: 



,distance,amount,duration
vehicle,,,
0,56251,[63],6017
1,71494,[48],7276
2,52456,[42],4790


In [36]:
stations = list()
for route in result['routes']:
    vehicle = list()
    for step in route["steps"]:
        vehicle.append(
            [
                step.get("job", "Depot"),  # Station ID
                step["arrival"],  # Arrival time
                step["arrival"] + step.get("service", 0), # Departure time
                step['location']

            ]
        )
    stations.append(vehicle)

In [37]:
# ### Vehicle 0
print("vehicle 0")
df_stations_0 = pd.DataFrame(stations[0], columns=["Station ID","Arrival", "Departure", "Location"])
df_stations_0['Arrival'] = pd.to_datetime(df_stations_0['Arrival'], unit='s')
df_stations_0['Departure'] = pd.to_datetime(df_stations_0['Departure'], unit='s')
display(df_stations_0)

# ### Vehicle 1
print("vehicle 1")
df_stations_1 = pd.DataFrame(stations[1], columns=["Station ID","Arrival", "Departure", "Location"])
df_stations_1['Arrival'] = pd.to_datetime(df_stations_1['Arrival'], unit='s')
df_stations_1['Departure'] = pd.to_datetime(df_stations_1['Departure'], unit='s')
display(df_stations_1)

# ### Vehicle 2
print("Vehicle 2")
df_stations_2 = pd.DataFrame(stations[2], columns=["Station ID","Arrival", "Departure", "Location"])
df_stations_2['Arrival'] = pd.to_datetime(df_stations_2['Arrival'], unit='s')
df_stations_2['Departure'] = pd.to_datetime(df_stations_2['Departure'], unit='s')
display(df_stations_2)

vehicle 0


,Station ID,Arrival,Departure,Location
0,Depot,2023-08-01 09:05:33,2023-08-01 09:05:33,"[10.777167937186457, 34.75093585415626]"
1,25,2023-08-01 09:12:33,2023-08-01 09:32:33,"[10.74809650624982, 34.774237905214754]"
2,16,2023-08-01 10:00:00,2023-08-01 10:20:00,"[10.688353807673876, 34.91540048817092]"
3,17,2023-08-01 10:29:38,2023-08-01 10:49:38,"[10.695331701581182, 34.901502378870596]"
4,18,2023-08-01 11:30:40,2023-08-01 11:50:40,"[10.692340608499148, 34.88487329127601]"
5,27,2023-08-01 13:22:09,2023-08-01 13:42:09,"[10.68571324173728, 34.88570922133694]"
6,11,2023-08-01 14:00:36,2023-08-01 14:20:36,"[10.746237722360776, 34.81089319621193]"
7,23,2023-08-01 14:27:50,2023-08-01 14:47:50,"[10.756098950994623, 34.78429181013371]"
8,22,2023-08-01 14:50:58,2023-08-01 15:10:58,"[10.741921926914513, 34.78145459148794]"
9,0,2023-08-01 15:25:32,2023-08-01 15:45:32,"[10.690784313871676, 34.72560838249331]"


vehicle 1


,Station ID,Arrival,Departure,Location
0,Depot,2023-08-01 07:05:55,2023-08-01 07:05:55,"[10.777167937186457, 34.75093585415626]"
1,21,2023-08-01 07:10:21,2023-08-01 07:30:21,"[10.77407319248187, 34.772385597224485]"
2,20,2023-08-01 08:00:00,2023-08-01 08:20:00,"[10.728120226859, 34.92590577160589]"
3,19,2023-08-01 08:36:16,2023-08-01 08:56:16,"[10.713401523744924, 34.93785039638581]"
4,10,2023-08-01 10:41:29,2023-08-01 11:01:29,"[10.778101192923131, 34.90252845030492]"
5,14,2023-08-01 11:17:58,2023-08-01 11:37:58,"[10.80278079524956, 34.86745209148853]"
6,12,2023-08-01 13:32:08,2023-08-01 13:52:08,"[10.836725620829991, 34.88868654935021]"
7,15,2023-08-01 14:02:03,2023-08-01 14:22:03,"[10.849263980861554, 34.850141338346596]"
8,13,2023-08-01 14:32:57,2023-08-01 14:52:57,"[10.869184362122288, 34.79773493678786]"
9,Depot,2023-08-01 17:20:00,2023-08-01 17:20:00,"[10.869184362122288, 34.79773493678786]"


Vehicle 2


,Station ID,Arrival,Departure,Location
0,Depot,2023-08-01 10:30:25,2023-08-01 10:30:25,"[10.777167937186457, 34.75093585415626]"
1,4,2023-08-01 10:36:02,2023-08-01 10:56:02,"[10.745409566146613, 34.741278025657735]"
2,5,2023-08-01 11:03:35,2023-08-01 11:23:35,"[10.705872964671062, 34.7537190552421]"
3,26,2023-08-01 11:36:10,2023-08-01 11:56:10,"[10.65825492025277, 34.796375318575556]"
4,24,2023-08-01 12:08:30,2023-08-01 12:28:30,"[10.71598880275222, 34.7759672614697]"
5,8,2023-08-01 12:32:05,2023-08-01 12:52:05,"[10.727988769186762, 34.76149022546576]"
6,6,2023-08-01 13:00:00,2023-08-01 13:20:00,"[10.744314825404208, 34.72678214420028]"
7,2,2023-08-01 13:28:24,2023-08-01 13:48:24,"[10.727914400426934, 34.68843091112631]"
8,3,2023-08-01 14:30:04,2023-08-01 14:50:04,"[10.68856401096503, 34.707124698904195]"
9,1,2023-08-01 15:01:51,2023-08-01 15:21:51,"[10.651203642067644, 34.66682292137911]"


### Filling the Requests Database

In [38]:
actual_location = [i.split('  ') for i in ideal_experience.db.location]

In [39]:
ideal_experience.db.id_livreur=None

In [40]:
d = [df_stations_0,df_stations_1,df_stations_2]
for r in range(3):
    df = d[r]
    for i in range(number_points):
        actual_location[i][0] = float(actual_location[i][0])
        actual_location[i][1] = float(actual_location[i][1])
        for j in range(len(df)):
            if haversine(df.Location[j][1],df.Location[j][0],actual_location[i][0],actual_location[i][1])<100:
                ideal_experience.db.id_livreur[i] = r
                ideal_experience.db.status[i] = 1

In [41]:
print("Requests After Filling")
display(ideal_experience.db)

Requests After Filling


,id,id_client,id_livreur,date,point,status,Open_From,Open_To,location,quantity,created_at
0,0,186,0,2023-08-01 16:00:00,28,1,2023-08-01 16:00:00,2023-08-01 20:00:00,34.72560838249331 10.690784313871676,15,2025-03-19 08:32:08
1,1,631,2,2023-08-01 13:00:00,8,1,2023-08-01 13:00:00,2023-08-01 19:00:00,34.66682292137911 10.651203642067644,9,2025-03-19 08:32:08
2,2,131,2,2023-08-01 14:00:00,16,1,2023-08-01 14:00:00,2023-08-01 15:00:00,34.68843091112631 10.727914400426934,1,2025-03-19 08:32:08
3,3,899,2,2023-08-01 12:00:00,32,1,2023-08-01 12:00:00,2023-08-01 18:00:00,34.707124698904195 10.68856401096503,3,2025-03-19 08:32:08
4,4,138,2,2023-08-01 06:00:00,88,1,2023-08-01 06:00:00,2023-08-01 16:00:00,34.741278025657735 10.745409566146613,10,2025-03-19 08:32:08
5,5,68,2,2023-08-01 06:00:00,1,1,2023-08-01 06:00:00,2023-08-01 13:00:00,34.7537190552421 10.705872964671062,1,2025-03-19 08:32:08
6,6,904,2,2023-08-01 12:00:00,85,1,2023-08-01 12:00:00,2023-08-01 13:00:00,34.72678214420028 10.744314825404208,7,2025-03-19 08:32:08
7,7,886,None,2023-08-01 19:00:00,74,0,2023-08-01 19:00:00,2023-08-01 20:00:00,34.7225763457473 10.745461573919977,8,2025-03-19 08:32:08
8,8,894,2,2023-08-01 10:00:00,77,1,2023-08-01 10:00:00,2023-08-01 19:00:00,34.76149022546576 10.727988769186762,4,2025-03-19 08:32:08
9,9,397,None,2023-08-01 19:00:00,65,0,2023-08-01 19:00:00,2023-08-01 20:00:00,34.73310966023656 10.745281805684796,2,2025-03-19 08:32:08


### Displaying unassigned Data Points 

In [42]:
non_assigned_points = [ideal_experience.db[i:i+1] for i in range(len(ideal_experience.db.id_livreur)) if ideal_experience.db.id_livreur[i] == None]

In [43]:
print('There Are/Is',len(non_assigned_points),'Unassigned Point(s): ')
print()
print('---------------------')
print()
print(non_assigned_points)

There Are/Is 4 Unassigned Point(s): 

---------------------

[   id  id_client id_livreur                date  point  status  \
7   7        886       None 2023-08-01 19:00:00     74       0   

            Open_From             Open_To  \
7 2023-08-01 19:00:00 2023-08-01 20:00:00   

                               location  quantity           created_at  
7  34.7225763457473  10.745461573919977         8  2025-03-19 08:32:08  ,    id  id_client id_livreur                date  point  status  \
9   9        397       None 2023-08-01 19:00:00     65       0   

            Open_From             Open_To  \
9 2023-08-01 19:00:00 2023-08-01 20:00:00   

                                location  quantity           created_at  
9  34.73310966023656  10.745281805684796         2  2025-03-19 08:32:08  ,     id  id_client id_livreur                date  point  status  \
28  28        212       None 2023-08-01 19:00:00     29       0   

             Open_From             Open_To  \
28 2023-08-01

# TESTS

In [41]:
ideal_experience.dc

,id,name,fname,phone,email,password,point,created_at
0,0,Mark,Lambert,+216 54690991,MarkLambert12@gmail.com,659055,45,2025-03-19 08:12:20
1,1,Shelby,Gray,+216 53600242,ShelbyGray8@gmail.com,585546,0,2025-03-19 08:12:20
2,2,Russell,Mccoy,+216 54953456,RussellMccoy5@gmail.com,833588,24,2025-03-19 08:12:20
3,3,Patricia,Harper,+216 74382106,PatriciaHarper9@gmail.com,133327,61,2025-03-19 08:12:20
4,4,Sarah,Humphrey,+216 72891642,SarahHumphrey10@gmail.com,292315,33,2025-03-19 08:12:20
...,...,...,...,...,...,...,...,...
995,995,John,Sanchez,+216 52072344,JohnSanchez7@gmail.com,1705902,79,2025-03-19 08:12:21
996,996,Lawrence,Gonzalez,+216 57418426,LawrenceGonzalez0@gmail.com,1002071,11,2025-03-19 08:12:21
997,997,Barry,Tanner,+216 73031072,BarryTanner9@gmail.com,1491511,95,2025-03-19 08:12:21
998,998,Colleen,Sheppard,+216 53817810,ColleenSheppard10@gmail.com,421605,66,2025-03-19 08:12:21


In [42]:
data_client = ideal_experience.dc

In [43]:
ideal_experience.db.created_at[0]

'2025-03-19 08:12:21'

In [44]:
data_client.id += 1

In [45]:
data_client = data_client.set_index('id')

In [46]:
data_client

,name,fname,phone,email,password,point,created_at
id,,,,,,,
1,Mark,Lambert,+216 54690991,MarkLambert12@gmail.com,659055,45,2025-03-19 08:12:20
2,Shelby,Gray,+216 53600242,ShelbyGray8@gmail.com,585546,0,2025-03-19 08:12:20
3,Russell,Mccoy,+216 54953456,RussellMccoy5@gmail.com,833588,24,2025-03-19 08:12:20
4,Patricia,Harper,+216 74382106,PatriciaHarper9@gmail.com,133327,61,2025-03-19 08:12:20
5,Sarah,Humphrey,+216 72891642,SarahHumphrey10@gmail.com,292315,33,2025-03-19 08:12:20
...,...,...,...,...,...,...,...
996,John,Sanchez,+216 52072344,JohnSanchez7@gmail.com,1705902,79,2025-03-19 08:12:21
997,Lawrence,Gonzalez,+216 57418426,LawrenceGonzalez0@gmail.com,1002071,11,2025-03-19 08:12:21
998,Barry,Tanner,+216 73031072,BarryTanner9@gmail.com,1491511,95,2025-03-19 08:12:21


In [47]:
for i in range(1,1001):
    data_client.created_at[i] = datetime.strptime(data_client.created_at[i], "%Y-%m-%d %H:%M:%S")

In [48]:
data_client.created_at[1]

datetime.datetime(2025, 3, 19, 8, 12, 20)

In [49]:
data_client['email'].value_counts()

JessicaPerry6@gmail.com       2
MarkLambert12@gmail.com       1
TerranceCollins6@gmail.com    1
CarlosWhitaker3@gmail.com     1
EricHuerta3@gmail.com         1
                             ..
MelissaWhite0@gmail.com       1
DavidFitzgerald4@gmail.com    1
NicholasShea2@gmail.com       1
WilliamHamilton5@gmail.com    1
JoelBrown10@gmail.com         1
Name: email, Length: 999, dtype: int64

In [50]:
data_client

,name,fname,phone,email,password,point,created_at
id,,,,,,,
1,Mark,Lambert,+216 54690991,MarkLambert12@gmail.com,659055,45,2025-03-19 08:12:20
2,Shelby,Gray,+216 53600242,ShelbyGray8@gmail.com,585546,0,2025-03-19 08:12:20
3,Russell,Mccoy,+216 54953456,RussellMccoy5@gmail.com,833588,24,2025-03-19 08:12:20
4,Patricia,Harper,+216 74382106,PatriciaHarper9@gmail.com,133327,61,2025-03-19 08:12:20
5,Sarah,Humphrey,+216 72891642,SarahHumphrey10@gmail.com,292315,33,2025-03-19 08:12:20
...,...,...,...,...,...,...,...
996,John,Sanchez,+216 52072344,JohnSanchez7@gmail.com,1705902,79,2025-03-19 08:12:21
997,Lawrence,Gonzalez,+216 57418426,LawrenceGonzalez0@gmail.com,1002071,11,2025-03-19 08:12:21
998,Barry,Tanner,+216 73031072,BarryTanner9@gmail.com,1491511,95,2025-03-19 08:12:21


In [51]:
data_client.to_csv('client_data.csv')

In [52]:
data_client.to_csv('client_Data.csv')

In [53]:
requests_data = ideal_experience.db

In [54]:
requests_data.id += 1

In [55]:
requests_data = requests_data.set_index('id')

In [56]:
requests_data = requests_data.drop(['Open_From','Open_To'],axis=1)

In [57]:
requests_data = requests_data.drop(['quantity'],axis=1)

In [58]:
requests_data.to_csv('requests_data.csv')

In [59]:
requests_data

,id_client,id_livreur,date,point,status,location,created_at
id,,,,,,,
1,393,0,2023-08-01 06:00:00,13,1,34.74943830250147 10.657460457742635,2025-03-19 08:12:21
2,379,1,2023-08-01 10:00:00,99,1,34.66395375834346 10.679456404364158,2025-03-19 08:12:21
3,763,0,2023-08-01 06:00:00,11,1,34.69656310060298 10.63639653772461,2025-03-19 08:12:21
4,149,1,2023-08-01 07:00:00,8,1,34.720798013495795 10.704413214132051,2025-03-19 08:12:21
5,67,1,2023-08-01 09:00:00,19,1,34.68702775204068 10.667903537782578,2025-03-19 08:12:21
6,846,None,2023-08-01 19:00:00,53,0,34.741310868897315 10.723884032858876,2025-03-19 08:12:21
7,510,None,2023-08-01 19:00:00,92,0,34.746884153318966 10.711481119736689,2025-03-19 08:12:21
8,447,1,2023-08-01 13:00:00,42,1,34.73447668172781 10.726555863894587,2025-03-19 08:12:21
9,578,None,2023-08-01 19:00:00,71,0,34.756839151218266 10.7112116076121,2025-03-19 08:12:21


In [ ]:
shuffled_df = colis_data.sample(frac=1).reset_index(drop=True)

In [ ]:
colis_data = shuffled_df

In [ ]:
colis_data.index += 1

In [ ]:
colis_data.to_csv('Colis_Data.csv')

In [ ]:
ideal_experience.dcolis[ideal_experience.dcolis.id_request == 0]

In [ ]:
colis_data[colis_data.id_request == 0]

In [ ]:
r = pd.read_csv("Requests_Data.csv",index_col="id")

## Recieving Data

In [ ]:
response = requests.get(url)

In [ ]:
response.json()

In [ ]:
for i in range(990,1001):
    response = requests.get(url + str(i))
    print(response.json())

### Requests Data

In [ ]:
requests_data.head()

### Colis Data

In [ ]:
df

In [ ]:
len(df)